In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Load dataset
# train_path = "/kaggle/input/off-dataset/train.json"
# test_path = "/kaggle/input/off-dataset/test.json"

train_path = r"D:\major project\data\train_final.json"
test_path = r"D:\major project\nepali-offensive-lang-detection-dataset\test.json"

train_df = pd.read_json(train_path)
test_df = pd.read_json(test_path)


# Show basic structure
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("\nSample data train:\n")
train_df.head()






Train shape: (5577, 4)
Test shape: (1450, 4)

Sample data train:



,ID,Comment,Label_Binary,Label_Multiclass
0,visewu,भूकने कूकूर ले कहीले टोकदैन भन्छ हो रैछ साला ध...,OFF,OR
1,hgpmyz,तेरी आमाको पसल मूजी कूकूरको मूत खाएसी जे पनी स...,OFF,OO
2,pwqyku,thukka ta machikne valu kun level ko hosh dekh...,OFF,OO
3,voaytw,ग्याँत अग्याँत मूजीहर्लाई ठूऊउउलो मूजी भन्न चा...,OFF,OO
4,mozgzm,गाईलाई गधा भनेर बीराेध गरेकाे छैन हाेला क्यारे,NOFF,NO


In [2]:
print(train_df.head())

       ID                                            Comment Label_Binary  \
0  visewu  भूकने कूकूर ले कहीले टोकदैन भन्छ हो रैछ साला ध...          OFF   
1  hgpmyz  तेरी आमाको पसल मूजी कूकूरको मूत खाएसी जे पनी स...          OFF   
2  pwqyku  thukka ta machikne valu kun level ko hosh dekh...          OFF   
3  voaytw  ग्याँत अग्याँत मूजीहर्लाई ठूऊउउलो मूजी भन्न चा...          OFF   
4  mozgzm     गाईलाई गधा भनेर बीराेध गरेकाे छैन हाेला क्यारे         NOFF   

  Label_Multiclass  
0               OR  
1               OO  
2               OO  
3               OO  
4               NO  


In [3]:
from sklearn.model_selection import train_test_split
# Create validation set (10–15% split from train set)
# train_df, val_df = train_test_split(
#     train_df, test_size=0.15, stratify=train_df["Label_Multiclass"], random_state=42
# )
val_df = pd.read_json('D:/major project/data/val_final.json')
print(f"\nNew train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")


New train size: 5577, Validation size: 620, Test size: 1450


In [21]:
# ! pip install gensim==4.3.1
# ! pip install emoji


In [1]:
import re
import emoji
import pandas as pd
import numpy as np
import regex as regex
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:


# --- Setup ---
nepali_stopwords = set([
    "र", "मा", "कि", "भने", "त", "छ", "हो", "लाई", "ले",
    "गरेको", "गर्छ", "गर्छन्", "हुन्", "गरे", "न", "नभएको"
])

dirghikaran_map = {
    # Vowel elongations / replacements (typical dirghikaran)
    "उ": "ऊ",
    "इ": "ई",
    "ऋ": "रि",  # Often replaced this way in Nepali
    "ए": "ऐ",   # More natural elongated vowel
    "अ": "आ",   # If you want to normalize short 'a' to long 'aa'

    # Remove zero-width joiner/non-joiner
    "\u200d": "",
    "\u200c": "",

    # Normalize punctuation marks
    "।": ".",
    "॥": ".",

    # Common vowel signs normalization (optional)
    "ि": "ी",
    "ु": "ू"
}


roman_stopwords = None  # Will be initialized below

# --- Helper Functions ---

def is_devanagari(text: str) -> bool:
    """Detect if text contains any Devanagari characters."""
    return bool(regex.search(r'\p{Devanagari}', text))

def devanagari_to_roman(text: str) -> str:
    try:
        return transliterate(text, sanscript.DEVANAGARI, sanscript.ITRANS)
    except Exception:
        return text

def roman_to_devanagari(text: str) -> str:
    try:
        return transliterate(text, sanscript.ITRANS, sanscript.DEVANAGARI)
    except Exception:
        return text

def normalize_dirghikaran(text: str) -> str:
    for original, replacement in dirghikaran_map.items():
        text = text.replace(original, replacement)
    return text


def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)         # Remove URLs
    text = re.sub(r"@\w+|#\w+", "", text)              # Remove mentions/hashtags
    text = re.sub(r"\d+", "", text)                    # Remove numbers
    text = emoji.replace_emoji(text, replace="")      # Remove emojis
    text = re.sub(r"[^\w\s]", "", text)                # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()           # Normalize whitespace
    return text

def clean_text_for_transformer(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # Remove URLs, emojis, and excessive whitespace
    text = re.sub(r"http\S+|www\S+", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"\s+", " ", text).strip()
    return text


def remove_stopwords_devanagari(text: str) -> str:
    """Remove Devanagari stopwords from text."""
    return ' '.join([word for word in text.split() if word not in nepali_stopwords])

def remove_stopwords_roman(text: str) -> str:
    """Remove Romanized stopwords from text."""
    global roman_stopwords
    if roman_stopwords is None:
        # Initialize roman_stopwords lazily to avoid dependency issues
        roman_stopwords = set([devanagari_to_roman(w) for w in nepali_stopwords])
    return ' '.join([word for word in text.split() if word not in roman_stopwords])

# --- Preprocessing Pipelines ---

def preprocess_for_ml_dl(text: str) -> str:
    """
    Preprocess input text for ML/GRU baselines:
    - Remove stopwords (script dependent)
    - Transliterate Devanagari → Roman
    - Clean Roman text
    """
    if not isinstance(text, str):
        return ""

    if is_devanagari(text):
        text = clean_text(text)
        text = remove_stopwords_devanagari(text)
        text = devanagari_to_roman(text)
    else:
        text = clean_text(text)
        text = remove_stopwords_roman(text)

    return text

def preprocess_for_transformer(text: str) -> str:
    """
    Preprocess input text for Transformer (XLM-Roberta):
    - Transliterate Roman → Devanagari if needed
    - Light cleaning only (no normalization, no punctuation stripping)
    """
    if not isinstance(text, str):
        return ""
    
    if not is_devanagari(text):
        text = roman_to_devanagari(text)

    text = clean_text_for_transformer(text)
    return text


# ----------- Apply Preprocessing to Dataset -----------

for df in [train_df, val_df, test_df]:
    # ML/GRU input: Romanized, cleaned, stopword-removed
    df["clean_comment"] = df["Comment"].apply(preprocess_for_ml_dl)
    df["tokens"] = df["clean_comment"].apply(str.split)

    # Transformer input: Devanagari, normalized, lightly cleaned
    df["transformer_input"] = df["Comment"].apply(preprocess_for_transformer)

# ----------- Inspect Preprocessing on Samples -----------
print("🔍 Sample Preprocessed Outputs:\n")
for idx, row in train_df[["Comment", "clean_comment", "transformer_input"]].head(10).iterrows():
    print(f"📝 Original       : {row['Comment']}")
    print(f"📊 ML/GRU Cleaned : {row['clean_comment']}")
    print(f"🤖 XLM-R Input    : {row['transformer_input']}")
    print("-" * 60)


🔍 Sample Preprocessed Outputs:

📝 Original       : भीखारी कांग्रेस का दलाल
📊 ML/GRU Cleaned : bhakhara kagarasa ka dalala
🤖 XLM-R Input    : भीखारी कांग्रेस का दलाल
------------------------------------------------------------
📝 Original       : फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठीटी फीट छ हाइट तेरो अर्को पाली देखी आफनो औकात हेरेर सीट ली बूझीस् अझै मूजी ईङलीस छाड्छ
📊 ML/GRU Cleaned : phalama halama daI tapaIka Taukala Chakaya bhanana ThaTa phaTa haiTa tara araka pala dakha Aphana aukata harara saTa la bajhasa ajha maja I~Nalasa ChaDaCha
🤖 XLM-R Input    : फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठीटी फीट छ हाइट तेरो अर्को पाली देखी आफनो औकात हेरेर सीट ली बूझीस् अझै मूजी ईङलीस छाड्छ
------------------------------------------------------------
📝 Original       : हैट बिदेस मा नि राजनिती पढेलेखेका मान्छे त झन positive खराव हुन्छ्न होला किन त सहिला सोधेर मार्‍यो
📊 ML/GRU Cleaned : haTa badasa ma rajanata paDhalakhaka manaCha jhana positive kharava hanaChana hala kana sahala sa

In [24]:
train_df.head(2)

,ID,Comment,Label_Binary,Label_Multiclass,clean_comment,tokens,transformer_input
3670,pqlvty,भीखारी कांग्रेस का दलाल,OFF,OO,bhakhara kagarasa ka dalala,"[bhakhara, kagarasa, ka, dalala]",भीखारी कांग्रेस का दलाल
3587,dbtslw,फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठ...,OFF,OO,phalama halama daI tapaIka Taukala Chakaya bha...,"[phalama, halama, daI, tapaIka, Taukala, Chaka...",फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठ...


In [ ]:
# # ============================================================================
# # ENHANCED PREPROCESSING PIPELINE (TRANSLATION + EMOJI)
# # ============================================================================

# import re
# import emoji
# import regex
# from typing import Optional
# from deep_translator import GoogleTranslator
# from functools import lru_cache
# import logging

# logging.basicConfig(level=logging.WARNING)
# logger = logging.getLogger(__name__)


# # Enhanced Emoji Mapping
# EMOJI_TO_NEPALI = {
#     # ===== Laughter & Joy (Positive) =====
#     '😂': 'हाँसो', '🤣': 'ठूलो हाँसो', '😀': 'खुशी', '😁': 'खुशी', '😃': 'खुशी',
#     '😄': 'खुशी', '😅': 'नर्भस हाँसो', '😆': 'हाँसो', '😊': 'मुस्कान', '☺️': 'मुस्कान',
#     '😉': 'आँखा झिम्काउने', '🙂': 'मुस्कान', '🙃': 'उल्टो मुस्कान', '😌': 'शान्त',
#     '😍': 'माया', '🥰': 'माया', '😘': 'चुम्बन', '😗': 'चुम्बन', '😙': 'चुम्बन', '😚': 'चुम्बन',
#     '🤗': 'अँगालो', '🤩': 'चकित', '🥳': 'उत्सव', '🤤': 'लालसा',
    
#     # ===== Mocking & Sarcasm (Hate Indicator) =====
#     '😏': 'व्यंग्य', '😜': 'जिब्रो देखाउने', '😝': 'जिब्रो देखाउने', '😛': 'जिब्रो',
#     '🙄': 'आँखा घुमाउने', '😤': 'निराश', '😑': 'अभिव्यक्तिहीन', '😐': 'तटस्थ',
#     '🙃': 'व्यंग्यात्मक', '😬': 'तनाव', '🤨': 'शंकास्पद', '🤫': 'चुपचाप',
#     '🤭': 'हात छोप्ने', '🤥': 'झूठ', '😶': 'मौन',
    
#     # ===== Anger & Hate (CRITICAL for Detection) =====
#     '😠': 'रिस', '😡': 'ठूलो रिस', '🤬': 'गाली', '😈': 'खराब', '👿': 'खराब',
#     '💢': 'क्रोध', '🔪': 'हिंसा', '💣': 'हिंसा', '🗡️': 'तरवार', '⚔️': 'युद्ध',
#     '💥': 'विस्फोट', '🔫': 'बन्दुक', '🧨': 'विस्फोटक', '☠️': 'मृत्यु', '💀': 'खोपडी',
#     '👹': 'राक्षस', '👺': 'दानव', '🤡': 'जोकर', '🖤': 'कालो मन',
#     '😾': 'रिसाएको', '👊': 'मुक्का', '✊': 'मुक्का',
    
#     # ===== Offensive Gestures (Strong Hate Signal) =====
#     '🖕': 'अपमान', '👎': 'नकारात्मक', '👎🏻': 'नकारात्मक', '👎🏼': 'नकारात्मक',
#     '👎🏽': 'नकारात्मक', '👎🏾': 'नकारात्मक', '👎🏿': 'नकारात्मक',
#     '🖕🏻': 'अपमान', '🖕🏼': 'अपमान', '🖕🏽': 'अपमान', '🖕🏾': 'अपमान', '🖕🏿': 'अपमान',
    
#     # ===== Sadness & Crying =====
#     '😭': 'रुवाइ', '😢': 'रुवाइ', '😿': 'रुवाइ', '😔': 'उदास', '😞': 'उदास',
#     '😒': 'उदास', '😓': 'चिन्तित', '😟': 'चिन्तित', '😕': 'अलमलिएको',
#     '🙁': 'तल्लो मुख', '☹️': 'दुःखी', '😩': 'थकित', '😫': 'थकित',
#     '😖': 'भ्रमित', '😣': 'अडिग', '😥': 'निराश', '🥺': 'बिन्ती',
    
#     # ===== Fear & Shock =====
#     '😨': 'डर', '😰': 'चिन्तित पसिना', '😱': 'चिच्याउने', '😳': 'लजाउने',
#     '🤯': 'मन उडेको', '😵': 'चक्कर', '😲': 'चकित', '😯': 'छक्क',
    
#     # ===== Disgust & Contempt =====
#     '🤢': 'बान्ता', '🤮': 'बान्ता', '🤧': 'हाच्छ्यूँ', '😷': 'बिरामी',
#     '🤒': 'ज्वरो', '🤕': 'घाइते', '🥴': 'मात्तिएको', '😪': 'निद्रा',
    
#     # ===== Positive Gestures & Symbols =====
#     '👍': 'सकारात्मक', '👍🏻': 'सकारात्मक', '👍🏼': 'सकारात्मक', 
#     '👍🏽': 'सकारात्मक', '👍🏾': 'सकारात्मक', '👍🏿': 'सकारात्मक',
#     '👏': 'तालि', '🙌': 'उत्सव', '👌': 'ठीक छ', '🤝': 'हात मिलाउनु',
#     '🙏': 'नमस्कार', '🤲': 'प्रार्थना', '💪': 'शक्ति', '✌️': 'शान्ति',
    
#     # ===== Hearts & Love =====
#     '❤️': 'माया', '🧡': 'माया', '💛': 'माया', '💚': 'माया', '💙': 'माया',
#     '💜': 'माया', '🖤': 'कालो मन', '🤍': 'सेतो मन', '🤎': 'खैरो मन',
#     '❣️': 'माया', '💕': 'माया', '💞': 'माया', '💓': 'माया', '💗': 'माया',
#     '💖': 'माया', '💘': 'माया', '💝': 'माया', '💔': 'टुटेको मन',
    
#     # ===== Symbols & Objects =====
#     '🔥': 'आगो', '💯': 'पूर्ण', '💢': 'क्रोध', '💨': 'हावा', '💫': 'चमक',
#     '⭐': 'तारा', '✨': 'चमक', '🌟': 'चम्किलो तारा', '💥': 'विस्फोट',
#     '🚫': 'निषेध', '⛔': 'प्रवेश निषेध', '❌': 'रद्द', '❎': 'गलत',
    
#     # ===== People & Relationships =====
#     '👫': 'जोडी', '👬': 'पुरुष जोडी', '👭': 'महिला जोडी', '👨‍👩‍👧‍👦': 'परिवार',
#     '👶': 'बच्चा', '👦': 'केटा', '👧': 'केटी', '👨': 'पुरुष', '👩': 'महिला',
#     '👴': 'बूढो', '👵': 'बूढी', '🧒': 'बालक', '👱': 'गोरो', '🧔': 'दाह्री',
    
#     # ===== Country & Culture =====
#     '🇳🇵': 'नेपाल', '🇮🇳': 'भारत', '🇵🇰': 'पाकिस्तान', '🇧🇩': 'बंगलादेश',
#     '🇨🇳': 'चीन', '🇺🇸': 'अमेरिका', '🏴': 'झण्डा',
    
#     # ===== Animals (sometimes used in hate) =====
#     '🐕': 'कुकुर', '🐖': 'सुँगुर', '🐀': 'मुसा', '🐍': 'सर्प', '🦂': 'बिच्छी',
#     '🐒': 'बाँदर', '🐵': 'बाँदर अनुहार', '🦍': 'गोरिल्ला', '🐗': 'जङ्गली सुँगुर',
    
#     # ===== Other Common =====
#     '🤔': 'सोच', '🧐': 'अनुसन्धान', '😴': 'सुत्ने', '💩': 'मल',
#     '👻': 'भूत', '🤖': 'रोबोट', '👽': 'विदेशी', '🎭': 'मुखौटा',
# }

# DIRGHIKARAN_MAP = {
#     "\u200d": "", "\u200c": "",
#     "।": ".", "॥": ".",
# }


# class CachedNepaliTranslator:
#     """Translator with caching"""
    
#     def __init__(self, cache_size: int = 2000):
#         self.translator = GoogleTranslator(source='en', target='ne')
#         self._translate_cached = lru_cache(maxsize=cache_size)(self._translate_single)
    
#     def _translate_single(self, text: str) -> str:
#         if not text or not text.strip():
#             return ""
#         try:
#             result = self.translator.translate(text.strip())
#             return result if result else text
#         except Exception as e:
#             logger.error(f"Translation failed: {str(e)}")
#             raise
    
#     def translate(self, text: str, fallback_to_original: bool = True) -> str:
#         if not text or not text.strip():
#             return ""
#         try:
#             return self._translate_cached(text.strip())
#         except Exception as e:
#             if fallback_to_original:
#                 logger.warning(f"Translation failed, using original: {str(e)}")
#                 return text
#             raise


# def devanagari_ratio(text: str) -> float:
#     """Calculate Devanagari character ratio"""
#     letters = regex.findall(r"\p{L}", text)
#     if not letters:
#         return 0.0
#     dev = regex.findall(r"\p{Devanagari}", text)
#     return len(dev) / len(letters)


# def is_mostly_devanagari(text: str, threshold: float = 0.6) -> bool:
#     """Check if text is mostly Devanagari"""
#     return devanagari_ratio(text) >= threshold


# def is_mostly_english(text: str) -> bool:
#     """Check if text is mostly English"""
#     letters = regex.findall(r"\p{L}", text)
#     if not letters:
#         return False
#     latin = regex.findall(r"[a-zA-Z]", text)
#     return len(latin) / len(letters) > 0.8


# def replace_emojis_semantic(text: str) -> str:
#     """Replace emojis with Nepali text"""
#     for emoji_char, nepali_text in EMOJI_TO_NEPALI.items():
#         text = text.replace(emoji_char, f" {nepali_text} ")
#     text = emoji.replace_emoji(text, replace=" ")
#     return text


# def normalize_devanagari(text: str) -> str:
#     """Normalize Devanagari characters"""
#     for k, v in DIRGHIKARAN_MAP.items():
#         text = text.replace(k, v)
#     return text


# def clean_text_basic(text: str) -> str:
#     """Basic text cleaning"""
#     text = text.lower()
#     text = re.sub(r"http\S+|www\S+", "", text)
#     text = re.sub(r"@\w+", "", text)
#     text = re.sub(r"#(\w+)", r"\1", text)
#     text = re.sub(r"\s+", " ", text).strip()
#     return text


# class HateSpeechPreprocessor:
#     """
#     Unified preprocessing - ALL inputs to Devanagari
#     """
    
#     def __init__(self, cache_size: int = 2000, translate_english: bool = True):
#         self.translate_english = translate_english
#         self.translator = CachedNepaliTranslator(cache_size) if translate_english else None
    
#     def preprocess(self, text: str, verbose: bool = False) -> str:
#         """
#         Main preprocessing - converts everything to Devanagari
#         """
#         if not isinstance(text, str) or not text.strip():
#             return ""
        
#         # Step 1: Replace emojis with Nepali
#         text = replace_emojis_semantic(text)
#         if verbose:
#             print(f"After emoji: {text}")
        
#         # Step 2: Basic cleaning
#         text = clean_text_basic(text)
#         if verbose:
#             print(f"After cleaning: {text}")
        
#         # Step 3: Script detection
#         is_devanagari_text = is_mostly_devanagari(text)
#         is_english_text = is_mostly_english(text)
        
#         if verbose:
#             print(f"Devanagari: {is_devanagari_text}, English: {is_english_text}")
        
#         # Step 4: Convert to Devanagari
#         if is_devanagari_text:
#             # Already Devanagari: normalize
#             text = normalize_devanagari(text)
#             if verbose:
#                 print(f"Normalized: {text}")
        
#         elif self.translate_english:
#             # Translate to Nepali (Devanagari)
#             try:
#                 text = self.translator.translate(text, fallback_to_original=True)
#                 text = normalize_devanagari(text)
#                 if verbose:
#                     print(f"Translated: {text}")
#             except Exception as e:
#                 logger.warning(f"Translation failed: {e}")
#                 text = normalize_devanagari(text)
        
#         else:
#             text = normalize_devanagari(text)
        
#         # Final cleanup
#         text = re.sub(r"\s+", " ", text).strip()
#         return text
    
#     def preprocess_batch(self, texts: list) -> list:
#         """Batch preprocessing"""
#         return [self.preprocess(text) for text in texts]


# # Initialize global preprocessor
# preprocessor = HateSpeechPreprocessor(cache_size=2000, translate_english=True)

# print("✓ Enhanced preprocessing pipeline loaded (Translation + Emoji)")


✓ Enhanced preprocessing pipeline loaded (Translation + Emoji)


In [ ]:
# # ============================================================================
# # JSON PREPROCESSING FOR NEPALI HATE SPEECH DATASET
# # (MATCHES YOUR ACTUAL SCHEMA) + TQDM
# # ============================================================================

# import os
# import json
# from typing import List, Dict
# from tqdm import tqdm

# # ---------------------------------------------------------------------------
# # CONFIG
# # ---------------------------------------------------------------------------

# INPUT_DIR = r"D:\major project\data"  # contains train_final.json, val_final.json, test.json
# OUTPUT_DIR = "translate_processed"

# FILES = {
#     "train": "train_final.json",
#     "val": "val_final.json",
#     "test": "test.json",
# }

# TEXT_KEY = "Comment"
# ID_KEY = "ID"
# BINARY_LABEL_KEY = "Label_Binary"
# MULTI_LABEL_KEY = "Label_Multiclass"

# os.makedirs(OUTPUT_DIR, exist_ok=True)

# # ---------------------------------------------------------------------------
# # JSON HELPERS
# # ---------------------------------------------------------------------------

# def load_json(path: str) -> List[Dict]:
#     with open(path, "r", encoding="utf-8") as f:
#         return json.load(f)


# def save_json(data: List[Dict], path: str) -> None:
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=2)

# # ---------------------------------------------------------------------------
# # PREPROCESSING
# # ---------------------------------------------------------------------------

# def preprocess_records(
#     records: List[Dict],
#     preprocessor: HateSpeechPreprocessor,
#     split_name: str
# ) -> List[Dict]:
#     processed = []

#     for item in tqdm(records, desc=f"🔄 Preprocessing {split_name}", unit="sample"):
#         raw_text = item.get(TEXT_KEY, "")

#         cleaned_text = preprocessor.preprocess(raw_text)

#         processed.append({
#             ID_KEY: item.get(ID_KEY),
#             TEXT_KEY: cleaned_text,
#             BINARY_LABEL_KEY: item.get(BINARY_LABEL_KEY),
#             MULTI_LABEL_KEY: item.get(MULTI_LABEL_KEY),
#         })

#     return processed

# # ---------------------------------------------------------------------------
# # MAIN PIPELINE
# # ---------------------------------------------------------------------------

# for split, filename in FILES.items():
#     input_path = os.path.join(INPUT_DIR, filename)
#     output_path = os.path.join(OUTPUT_DIR, f"{split}_processed.json")

#     print(f"\n📥 Loading {split} split...")
#     data = load_json(input_path)

#     processed_data = preprocess_records(
#         records=data,
#         preprocessor=preprocessor,
#         split_name=split.upper()
#     )

#     save_json(processed_data, output_path)

#     print(f"✓ Saved {len(processed_data)} samples → {output_path}")

# print("\n✅ All JSON splits processed and saved in 'translate_processed/'")



📥 Loading train split...


🔄 Preprocessing TRAIN:  19%|█▉        | 1085/5577 [13:19<55:10,  1.36sample/s]  


KeyboardInterrupt: 

In [22]:
"""
Enhanced Preprocessing Pipeline for Nepali Hate Speech Classification
CRITICAL FIX: Script detection BEFORE emoji handling
WITH EMOJI INTELLIGENCE: Extract features from emoji patterns
- Romanized Nepali → Transliteration to Devanagari
- English → Translation to Devanagari
- Emojis replaced AFTER script conversion WITH feature extraction
"""

import re
import emoji
import regex
from typing import Literal, Optional, Tuple, Dict
from deep_translator import GoogleTranslator
from functools import lru_cache
import logging

# Setup logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# Try to import transliteration (optional)
try:
    from indic_transliteration import sanscript
    from indic_transliteration.sanscript import transliterate
    TRANSLITERATION_AVAILABLE = True
except ImportError:
    TRANSLITERATION_AVAILABLE = False
    logger.warning("indic_transliteration not available. Transliteration disabled.")


# ============================================================================
# ENHANCED EMOJI MAPPING (Semantic & Context-Aware)
# ============================================================================

EMOJI_TO_NEPALI = {
    # ===== Laughter & Joy (Positive) =====
    '😂': 'हाँसो', '🤣': 'ठूलो हाँसो', '😀': 'खुशी', '😁': 'खुशी', '😃': 'खुशी',
    '😄': 'खुशी', '😅': 'नर्भस हाँसो', '😆': 'हाँसो', '😊': 'मुस्कान', '☺️': 'मुस्कान',
    '😉': 'आँखा झिम्काउने', '🙂': 'मुस्कान', '🙃': 'उल्टो मुस्कान', '😌': 'शान्त',
    '😍': 'माया', '🥰': 'माया', '😘': 'चुम्बन', '😗': 'चुम्बन', '😙': 'चुम्बन', '😚': 'चुम्बन',
    '🤗': 'अँगालो', '🤩': 'चकित', '🥳': 'उत्सव', '🤤': 'लालसा',
    
    # ===== Mocking & Sarcasm (Hate Indicator) =====
    '😏': 'व्यंग्य', '😜': 'जिब्रो देखाउने', '😝': 'जिब्रो देखाउने', '😛': 'जिब्रो',
    '🙄': 'आँखा घुमाउने', '😤': 'निराश', '😑': 'अभिव्यक्तिहीन', '😐': 'तटस्थ',
    '🙃': 'व्यंग्यात्मक', '😬': 'तनाव', '🤨': 'शंकास्पद', '🤫': 'चुपचाप',
    '🤭': 'हात छोप्ने', '🤥': 'झूठ', '😶': 'मौन',
    
    # ===== Anger & Hate (CRITICAL for Detection) =====
    '😠': 'रिस', '😡': 'ठूलो रिस', '🤬': 'गाली', '😈': 'खराब', '👿': 'खराब',
    '💢': 'क्रोध', '🔪': 'हिंसा', '💣': 'हिंसा', '🗡️': 'तरवार', '⚔️': 'युद्ध',
    '💥': 'विस्फोट', '🔫': 'बन्दुक', '🧨': 'विस्फोटक', '☠️': 'मृत्यु', '💀': 'खोपडी',
    '👹': 'राक्षस', '👺': 'दानव', '🤡': 'जोकर', '🖤': 'कालो मन',
    '😾': 'रिसाएको', '👊': 'मुक्का', '✊': 'मुक्का',
    
    # ===== Offensive Gestures (Strong Hate Signal) =====
    '🖕': 'अपमान', '👎': 'नकारात्मक', '👎🏻': 'नकारात्मक', '👎🏼': 'नकारात्मक',
    '👎🏽': 'नकारात्मक', '👎🏾': 'नकारात्मक', '👎🏿': 'नकारात्मक',
    '🖕🏻': 'अपमान', '🖕🏼': 'अपमान', '🖕🏽': 'अपमान', '🖕🏾': 'अपमान', '🖕🏿': 'अपमान',
    
    # ===== Sadness & Crying =====
    '😭': 'रुवाइ', '😢': 'रुवाइ', '😿': 'रुवाइ', '😔': 'उदास', '😞': 'उदास',
    '😒': 'उदास', '😓': 'चिन्तित', '😟': 'चिन्तित', '😕': 'अलमलिएको',
    '🙁': 'तल्लो मुख', '☹️': 'दुःखी', '😩': 'थकित', '😫': 'थकित',
    '😖': 'भ्रमित', '😣': 'अडिग', '😥': 'निराश', '🥺': 'बिन्ती',
    
    # ===== Fear & Shock =====
    '😨': 'डर', '😰': 'चिन्तित पसिना', '😱': 'चिच्याउने', '😳': 'लजाउने',
    '🤯': 'मन उडेको', '😵': 'चक्कर', '😲': 'चकित', '😯': 'छक्क',
    
    # ===== Disgust & Contempt =====
    '🤢': 'बान्ता', '🤮': 'बान्ता', '🤧': 'हाच्छ्यूँ', '😷': 'बिरामी',
    '🤒': 'ज्वरो', '🤕': 'घाइते', '🥴': 'मात्तिएको', '😪': 'निद्रा',
    
    # ===== Positive Gestures & Symbols =====
    '👍': 'सकारात्मक', '👍🏻': 'सकारात्मक', '👍🏼': 'सकारात्मक', 
    '👍🏽': 'सकारात्मक', '👍🏾': 'सकारात्मक', '👍🏿': 'सकारात्मक',
    '👏': 'तालि', '🙌': 'उत्सव', '👌': 'ठीक छ', '🤝': 'हात मिलाउनु',
    '🙏': 'नमस्कार', '🤲': 'प्रार्थना', '💪': 'शक्ति', '✌️': 'शान्ति',
    
    # ===== Hearts & Love =====
    '❤️': 'माया', '🧡': 'माया', '💛': 'माया', '💚': 'माया', '💙': 'माया',
    '💜': 'माया', '🖤': 'कालो मन', '🤍': 'सेतो मन', '🤎': 'खैरो मन',
    '❣️': 'माया', '💕': 'माया', '💞': 'माया', '💓': 'माया', '💗': 'माया',
    '💖': 'माया', '💘': 'माया', '💝': 'माया', '💔': 'टुटेको मन',
    
    # ===== Symbols & Objects =====
    '🔥': 'आगो', '💯': 'पूर्ण', '💢': 'क्रोध', '💨': 'हावा', '💫': 'चमक',
    '⭐': 'तारा', '✨': 'चमक', '🌟': 'चम्किलो तारा', '💥': 'विस्फोट',
    '🚫': 'निषेध', '⛔': 'प्रवेश निषेध', '❌': 'रद्द', '❎': 'गलत',
    
    # ===== People & Relationships =====
    '👫': 'जोडी', '👬': 'पुरुष जोडी', '👭': 'महिला जोडी', '👨‍👩‍👧‍👦': 'परिवार',
    '👶': 'बच्चा', '👦': 'केटा', '👧': 'केटी', '👨': 'पुरुष', '👩': 'महिला',
    '👴': 'बूढो', '👵': 'बूढी', '🧒': 'बालक', '👱': 'गोरो', '🧔': 'दाह्री',
    
    # ===== Country & Culture =====
    '🇳🇵': 'नेपाल', '🇮🇳': 'भारत', '🇵🇰': 'पाकिस्तान', '🇧🇩': 'बंगलादेश',
    '🇨🇳': 'चीन', '🇺🇸': 'अमेरिका', '🏴': 'झण्डा',
    
    # ===== Animals (sometimes used in hate) =====
    '🐕': 'कुकुर', '🐖': 'सुँगुर', '🐀': 'मुसा', '🐍': 'सर्प', '🦂': 'बिच्छी',
    '🐒': 'बाँदर', '🐵': 'बाँदर अनुहार', '🦍': 'गोरिल्ला', '🐗': 'जङ्गली सुँगुर',
    
    # ===== Other Common =====
    '🤔': 'सोच', '🧐': 'अनुसन्धान', '😴': 'सुत्ने', '💩': 'मल',
    '👻': 'भूत', '🤖': 'रोबोट', '👽': 'विदेशी', '🎭': 'मुखौटा',
}

# Emoji categories for INTELLIGENT FEATURE EXTRACTION
HATE_RELATED_EMOJIS = {
    # Anger & Violence
    '😠', '😡', '🤬', '😈', '👿', '💢', '👊', '✊',
    # Weapons & Violence
    '🔪', '💣', '🗡️', '⚔️', '💥', '🔫', '🧨', '☠️', '💀',
    # Offensive Gestures
    '🖕', '🖕🏻', '🖕🏼', '🖕🏽', '🖕🏾', '🖕🏿',
    '👎', '👎🏻', '👎🏼', '👎🏽', '👎🏾', '👎🏿',
    # Monsters & Evil
    '👹', '👺', '🤡',
    # Negative Hearts
    '🖤', '💔',
    # Animals (often used derogatorily)
    '🐕', '🐖', '🐀', '🐍', '🦂', '🐒', '🐵', '🦍', '🐗',
    # Other
    '💩', '😾',
}

MOCKERY_EMOJIS = {
    # Sarcasm
    '😏', '😜', '😝', '😛', '🙄', '😤', '🙃',
    # Contempt
    '😑', '😐', '😬', '🤨', '🤫', '🤭', '🤥',
    # Mockery
    '🤡', '👻', '🎭',
}

POSITIVE_EMOJIS = {
    # Happy faces
    '😊', '😀', '😁', '😃', '😄', '☺️', '🙂', '😌', '🥰', '😍',
    # Love & Hearts
    '❤️', '🧡', '💛', '💚', '💙', '💜', '🤍', '🤎',
    '💕', '💞', '💓', '💗', '💖', '💘', '💝', '❣️',
    # Positive gestures
    '👍', '👍🏻', '👍🏼', '👍🏽', '👍🏾', '👍🏿',
    '🙏', '👏', '🙌', '👌', '🤝', '✌️',
    # Celebration
    '🥳', '🎉', '🎊', '⭐', '✨', '🌟',
}

SADNESS_EMOJIS = {
    '😭', '😢', '😿', '😔', '😞', '😒', '😓', '😟', '😕',
    '🙁', '☹️', '😩', '😫', '😖', '😣', '😥', '🥺',
}

FEAR_EMOJIS = {
    '😨', '😰', '😱', '😳', '🤯', '😵', '😲', '😯',
}

DISGUST_EMOJIS = {
    '🤢', '🤮', '🤧', '😷', '🤒', '🤕', '🥴',
}


# ============================================================================
# EMOJI INTELLIGENCE - FEATURE EXTRACTION
# ============================================================================

def extract_emoji_features(text: str) -> Dict[str, int]:
    """
    Extract COMPREHENSIVE emoji-based semantic features for hate speech detection.
    
    PATTERN 1 (RECOMMENDED): Emoji-derived feature flags
    These features can be:
    - Concatenated to classifier head (DL)
    - Used as extra features (ML)
    - Logged for analysis
    
    Returns:
        Dictionary with emoji features (15 features total):
        Binary flags (6):
        - has_hate_emoji, has_mockery_emoji, has_positive_emoji
        - has_sadness_emoji, has_fear_emoji, has_disgust_emoji
        
        Count features (6):
        - hate_emoji_count, mockery_emoji_count, positive_emoji_count
        - sadness_emoji_count, fear_emoji_count, disgust_emoji_count
        
        Derived features (3):
        - total_emoji_count
        - hate_to_positive_ratio
        - has_mixed_sentiment
    """
    # Extract all emojis from text
    emojis_found = [c for c in text if c in emoji.EMOJI_DATA]
    
    # Count by category
    hate_count = sum(1 for e in emojis_found if e in HATE_RELATED_EMOJIS)
    mockery_count = sum(1 for e in emojis_found if e in MOCKERY_EMOJIS)
    positive_count = sum(1 for e in emojis_found if e in POSITIVE_EMOJIS)
    sadness_count = sum(1 for e in emojis_found if e in SADNESS_EMOJIS)
    fear_count = sum(1 for e in emojis_found if e in FEAR_EMOJIS)
    disgust_count = sum(1 for e in emojis_found if e in DISGUST_EMOJIS)
    
    return {
        # ===== Binary flags (presence/absence) =====
        'has_hate_emoji': 1 if hate_count > 0 else 0,
        'has_mockery_emoji': 1 if mockery_count > 0 else 0,
        'has_positive_emoji': 1 if positive_count > 0 else 0,
        'has_sadness_emoji': 1 if sadness_count > 0 else 0,
        'has_fear_emoji': 1 if fear_count > 0 else 0,
        'has_disgust_emoji': 1 if disgust_count > 0 else 0,
        
        # ===== Count features =====
        'hate_emoji_count': hate_count,
        'mockery_emoji_count': mockery_count,
        'positive_emoji_count': positive_count,
        'sadness_emoji_count': sadness_count,
        'fear_emoji_count': fear_count,
        'disgust_emoji_count': disgust_count,
        'total_emoji_count': len(emojis_found),
        
        # ===== Derived features =====
        'hate_to_positive_ratio': hate_count / max(positive_count, 1),
        'has_mixed_sentiment': 1 if (hate_count > 0 and positive_count > 0) else 0,
    }


def inject_emoji_tags(text: str, emoji_features: Dict[str, int]) -> str:
    """
    Inject CONTROLLED emoji-derived semantic tags at the beginning of text.

    DESIGN PRINCIPLES:
    - Low number of tags (avoid noise)
    - Emotion-level abstraction (not raw emojis)
    - Compatible with Transformer tokenization
    - Deterministic mapping from extracted features

    Tags are injected ONLY if present.
    """

    tags = []

    # ===============================
    # PRIMARY SENTIMENT TAGS
    # ===============================
    if emoji_features.get('has_hate_emoji', 0):
        tags.append('<EMO_HATE>')

    if emoji_features.get('has_mockery_emoji', 0):
        tags.append('<EMO_MOCKERY>')

    if emoji_features.get('has_positive_emoji', 0):
        tags.append('<EMO_POSITIVE>')

    # ===============================
    # SECONDARY EMOTION TAGS
    # ===============================
    if emoji_features.get('has_sadness_emoji', 0):
        tags.append('<EMO_SADNESS>')

    if emoji_features.get('has_fear_emoji', 0):
        tags.append('<EMO_FEAR>')

    if emoji_features.get('has_disgust_emoji', 0):
        tags.append('<EMO_DISGUST>')

    # ===============================
    # DERIVED / COMPOSITE SIGNALS
    # ===============================
    if emoji_features.get('has_mixed_sentiment', 0):
        tags.append('<EMO_MIXED>')

    # High-intensity hate (multiple hate emojis)
    if emoji_features.get('hate_emoji_count', 0) >= 2:
        tags.append('<EMO_HATE_STRONG>')

    # ===============================
    # FINAL OUTPUT
    # ===============================
    if tags:
        return ' '.join(tags) + ' ' + text

    return text



# ============================================================================
# SCRIPT DETECTION (IGNORES EMOJIS)
# ============================================================================

ScriptType = Literal["devanagari", "romanized_nepali", "english", "mixed", "other"]


def remove_emojis_for_detection(text: str) -> str:
    """
    Remove emojis temporarily for script detection
    (emojis will be replaced with Nepali text later)
    """
    return emoji.replace_emoji(text, replace="")


def detect_script_type(text: str) -> Tuple[ScriptType, dict]:
    """
    Detect the dominant script type IGNORING emojis.
    
    CRITICAL: Emojis are removed before detection to get accurate script classification.
    
    Returns:
        Tuple of (script_type, detection_details)
    """
    if not text or not text.strip():
        return "other", {"confidence": 0.0, "reason": "empty_text"}
    
    # CRITICAL: Remove emojis before script detection
    text_no_emoji = remove_emojis_for_detection(text)
    
    if not text_no_emoji.strip():
        # Only emojis
        return "other", {"confidence": 0.5, "reason": "emoji_only"}
    
    # Count different character types (WITHOUT emojis)
    letters = regex.findall(r"\p{L}", text_no_emoji)
    letter_count = len(letters)
    
    if letter_count == 0:
        return "other", {"confidence": 0.0, "reason": "no_letters"}
    
    # Count Devanagari characters
    devanagari_chars = regex.findall(r"\p{Devanagari}", text_no_emoji)
    dev_count = len(devanagari_chars)
    dev_ratio = dev_count / letter_count
    
    # Count Latin characters
    latin_chars = regex.findall(r"[a-zA-Z]", text_no_emoji)
    latin_count = len(latin_chars)
    latin_ratio = latin_count / letter_count
    
    # Romanized Nepali indicators (EXPANDED)
    romanized_nepali_patterns = [
        # Common words
        r'\b[xX]u\b', r'\b[xX]um?\b', r'\bhajur\b', r'\bdai\b', r'\bbhai\b', r'\bdidi\b',
        r'\bbahini\b', r'\bsanghai\b', r'\bsunu\b', r'\bhera\b', r'\bsun\b',
        
        # Particles & Postpositions
        r'\bko\b', r'\bki\b', r'\bka\b', r'\bho\b', r'\btyo\b', r'\byo\b', r'\bta\b',
        r'\bma\b', r'\bma?i\b', r'\bla[ie]?\b', r'\bnai?\b', r'\bpani\b', r'\bni\b',
        
        # Verbs
        r'\bhun[ae]\b', r'\bhunchha\b', r'\bhunuhunchha\b', r'\bgar\w+\b', r'\bgarna\b',
        r'\bx[ao]\b', r'\bxa\b', r'\bxan\b', r'\bxaina\b', r'\bxu\b',
        r'\bchain\b', r'\bchaina\b', r'\bthiy[oe]\b', r'\bhola\b', r'\bhos\b',
        r'\bbhan\w*\b', r'\bbol\w*\b', r'\bher\w*\b',
        
        # Common adjectives/states
        r'\bkh[ou]s[hi]?\b', r'\bkhusi\b', r'\bkhushi\b', r'\bramro\b', r'\bnaramro\b',
        r'\bthulo\b', r'\bsano\b', r'\brasilo\b', r'\bmitho\b', r'\btikhi\b',
        r'\bdherei\b', r'\baliali\b', r'\bastai\b', r'\blastai\b',
        
        # Question words
        r'\bkina\b', r'\bkasari\b', r'\bkahile\b', r'\bkaha[n]?\b', r'\bke\b', r'\bko\b',
        
        # Pronouns
        r'\bma\b', r'\btimi\b', r'\btapai\b', r'\buha\b', r'\buni\b', r'\byini\b',
        r'\bmero\b', r'\btimro\b', r'\buhako\b', r'\buniko\b', r'\bhamro\b',
        
        # Common nouns
        r'\bmanxe\b', r'\bmanchhe\b', r'\bmanche\b', r'\bharu\b', r'\bdes[ha]?\b',
        r'\bgha?r\b', r'\bthau\b', r'\bsamay\b', r'\bbela\b',
        
        # Nepali-specific endings (transliterated)
        r'\w+[ae]ko\b', r'\w+[ae]ki\b', r'\w+dai\b', r'\w+lai\b',
        r'\w+ma\b', r'\w+xa\b', r'\w+hun[ae]\b', r'\w+thiyo\b',
    ]
    
    romanized_indicators = sum(1 for pattern in romanized_nepali_patterns 
                               if re.search(pattern, text_no_emoji, re.IGNORECASE))
    
    # Calculate Romanized Nepali score
    romanized_score = 0.0
    if latin_ratio > 0.5 and dev_ratio < 0.3:
        if romanized_indicators > 0:
            romanized_score = min(0.5 + (romanized_indicators * 0.15), 0.95)
        else:
            # Check for typical Romanized Nepali patterns
            romanized_patterns = re.findall(r'\b\w*[aeiou](?:h)?\b', text_no_emoji.lower())
            if any(word.endswith(('xu', 'ro', 'no', 'lo', 'ko', 'ho')) 
                   for word in romanized_patterns):
                romanized_score = 0.4
            else:
                romanized_score = 0.3
    
    # English indicators (EXPANDED)
    english_indicators = [
        # Articles & Determiners
        'the', 'a', 'an', 'this', 'that', 'these', 'those', 'some', 'any', 'all', 'every',
        
        # Pronouns
        'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them',
        'my', 'your', 'his', 'her', 'its', 'our', 'their', 'mine', 'yours', 'ours', 'theirs',
        'myself', 'yourself', 'himself', 'herself', 'itself', 'ourselves', 'themselves',
        
        # Common verbs (be, have, do)
        'is', 'am', 'are', 'was', 'were', 'be', 'been', 'being',
        'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'done',
        'will', 'would', 'shall', 'should', 'can', 'could', 'may', 'might', 'must',
        
        # Common verbs (action)
        'get', 'got', 'go', 'went', 'gone', 'make', 'made', 'take', 'took', 'taken',
        'come', 'came', 'see', 'saw', 'seen', 'know', 'knew', 'known', 'say', 'said',
        'tell', 'told', 'think', 'thought', 'give', 'gave', 'given', 'find', 'found',
        
        # Question words
        'what', 'which', 'who', 'whom', 'whose', 'when', 'where', 'why', 'how',
        
        # Prepositions
        'in', 'on', 'at', 'to', 'for', 'of', 'with', 'from', 'by', 'about', 'as',
        'into', 'through', 'over', 'under', 'after', 'before', 'between', 'among',
        
        # Conjunctions
        'and', 'or', 'but', 'so', 'yet', 'nor', 'because', 'if', 'when', 'while',
        'although', 'though', 'unless', 'since', 'until', 'where', 'whether',
        
        # Negations
        'not', 'no', 'never', 'none', 'nothing', 'nobody', 'nowhere', 'neither',
        
        # Common adjectives
        'good', 'bad', 'great', 'big', 'small', 'long', 'short', 'high', 'low',
        'old', 'new', 'young', 'early', 'late', 'right', 'wrong', 'true', 'false',
        'hot', 'cold', 'happy', 'sad', 'angry', 'nice', 'beautiful', 'ugly',
        
        # Sentiment words (hate speech relevant)
        'hate', 'love', 'like', 'dislike', 'stupid', 'dumb', 'idiot', 'fool',
        'kill', 'die', 'dead', 'death', 'fuck', 'shit', 'ass', 'damn', 'hell',
        'worst', 'terrible', 'horrible', 'awful', 'disgusting', 'pathetic',
        
        # Common nouns
        'man', 'woman', 'people', 'person', 'thing', 'time', 'day', 'year',
        'way', 'work', 'life', 'world', 'country', 'place', 'home', 'hand',
        
        # Very & Adverbs
        'very', 'really', 'quite', 'too', 'so', 'just', 'only', 'even', 'also',
        'well', 'much', 'more', 'most', 'less', 'least', 'still', 'already',
    ]
    english_words = [w.lower() for w in re.findall(r'\b\w+\b', text_no_emoji)]
    english_count = sum(1 for w in english_words if w in english_indicators)
    english_ratio = english_count / len(english_words) if english_words else 0
    
    # Detection details
    details = {
        "devanagari_count": dev_count,
        "devanagari_ratio": dev_ratio,
        "latin_count": latin_count,
        "latin_ratio": latin_ratio,
        "romanized_indicators": romanized_indicators,
        "english_ratio": english_ratio,
        "letter_count": letter_count
    }
    
    # Decision logic
    if dev_ratio >= 0.8:
        return "devanagari", {**details, "confidence": dev_ratio, "reason": "dominant_devanagari"}
    
    elif dev_ratio >= 0.4:
        return "mixed", {**details, "confidence": 0.7, "reason": "mixed_with_devanagari"}
    
    elif romanized_score > 0.5 and dev_ratio < 0.2:
        return "romanized_nepali", {**details, "confidence": romanized_score, "reason": "romanized_nepali_detected"}
    
    elif english_ratio > 0.2 and romanized_score < 0.4:
        return "english", {**details, "confidence": min(english_ratio + 0.3, 0.9), "reason": "english_detected"}
    
    elif latin_ratio > 0.5 and romanized_score > 0.3:
        return "romanized_nepali", {**details, "confidence": romanized_score, "reason": "likely_romanized_nepali"}
    
    elif latin_ratio > 0.8:
        if english_ratio > 0.1:
            return "english", {**details, "confidence": 0.6, "reason": "likely_english"}
        else:
            return "romanized_nepali", {**details, "confidence": 0.5, "reason": "ambiguous_latin_script"}
    
    else:
        return "other", {**details, "confidence": 0.3, "reason": "insufficient_indicators"}


# ============================================================================
# CACHED TRANSLATOR
# ============================================================================

class CachedNepaliTranslator:
    """Production-ready translator with caching"""
    
    def __init__(self, cache_size: int = 2000):
        self.translator = GoogleTranslator(source='en', target='ne')
        self.cache_size = cache_size
        self._translate_cached = lru_cache(maxsize=cache_size)(self._translate_single)
    
    def _translate_single(self, text: str) -> str:
        if not text or not text.strip():
            return ""
        try:
            result = self.translator.translate(text.strip())
            return result if result else text
        except Exception as e:
            logger.error(f"Translation failed: {str(e)}")
            raise
    
    def translate(self, text: str, fallback_to_original: bool = True) -> str:
        if not text or not text.strip():
            return ""
        try:
            return self._translate_cached(text.strip())
        except Exception as e:
            if fallback_to_original:
                logger.warning(f"Translation failed, using original: {str(e)}")
                return text
            raise
    
    def get_cache_info(self) -> dict:
        cache_info = self._translate_cached.cache_info()
        return {
            'hits': cache_info.hits,
            'misses': cache_info.misses,
            'size': cache_info.currsize,
            'max_size': cache_info.maxsize,
            'hit_rate': cache_info.hits / (cache_info.hits + cache_info.misses) 
                       if (cache_info.hits + cache_info.misses) > 0 else 0.0
        }


# ============================================================================
# SCRIPT-SPECIFIC PROCESSING
# ============================================================================

def transliterate_romanized_nepali(text: str) -> str:
    """Transliterate Romanized Nepali to Devanagari using ITRANS"""
    if not TRANSLITERATION_AVAILABLE:
        logger.warning("Transliteration not available")
        return text
    
    try:
        result = transliterate(text, sanscript.ITRANS, sanscript.DEVANAGARI)
        return result if result else text
    except Exception as e:
        logger.warning(f"Transliteration failed: {e}")
        return text


def translate_latin_spans(text: str, translator: CachedNepaliTranslator) -> str:
    """
    Translate ONLY Latin word spans inside a Devanagari-dominant sentence.
    """
    def repl(match):
        latin_text = match.group(0)
        translated = translator.translate(latin_text, fallback_to_original=True)
        return f" {translated} "

    # Match continuous Latin words (including phrases)
    return re.sub(r"[A-Za-z][A-Za-z\s]{2,}", repl, text)


# ============================================================================
# EMOJI PROCESSING (APPLIED AFTER SCRIPT CONVERSION)
# ============================================================================

def replace_emojis_semantic(text: str, preserve_spacing: bool = True) -> str:
    """
    Replace emojis with Nepali text
    CALLED AFTER script conversion to Devanagari
    """
    for emoji_char, nepali_text in EMOJI_TO_NEPALI.items():
        if preserve_spacing:
            text = text.replace(emoji_char, f" {nepali_text} ")
        else:
            text = text.replace(emoji_char, nepali_text)
    
    # Remove any remaining unknown emojis
    text = emoji.replace_emoji(text, replace=" ")
    
    return text


# ============================================================================
# NORMALIZATION
# ============================================================================

DIRGHIKARAN_MAP = {
    "\u200d": "",  # Zero-width joiner
    "\u200c": "",  # Zero-width non-joiner
    "।": ".",      # Devanagari danda
    "॥": ".",      # Double danda
}


def normalize_devanagari(text: str) -> str:
    """Normalize Devanagari-specific characters"""
    for k, v in DIRGHIKARAN_MAP.items():
        text = text.replace(k, v)
    return text


def clean_text_basic(text: str) -> str:
    """Basic cleaning: URLs, mentions, whitespace"""
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#(\w+)", r"\1", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


# ============================================================================
# MAIN PREPROCESSING PIPELINE
# ============================================================================

class HateSpeechPreprocessor:
    """
    Unified preprocessing pipeline for Nepali hate speech classification
    WITH EMOJI INTELLIGENCE
    
    CRITICAL PIPELINE ORDER:
    1. EXTRACT emoji features (BEFORE replacement)
    2. Detect script type (emojis ignored for detection)
    3. Apply script-specific processing
    4. Replace emojis with Nepali text
    5. Normalize Devanagari
    
    ALL models receive Devanagari output + emoji features.
    """

    def __init__(
        self,
        model_type: Literal["xlmr", "mbert", "nepalibert"] = "xlmr",
        translate_english: bool = True,
        cache_size: int = 2000,
        use_emoji_tags: bool = False  # Pattern 2: inject tags
    ):
        """
        Args:
            model_type: Target model
            translate_english: Whether to translate English
            cache_size: Translation cache size
            use_emoji_tags: If True, inject <EMO_*> tags (Pattern 2)
                          If False, return features separately (Pattern 1, recommended)
        """
        self.model_type = model_type
        self.translate_english = translate_english
        self.use_emoji_tags = use_emoji_tags
        self.translator = CachedNepaliTranslator(cache_size) if translate_english else None

    def preprocess(self, text: str, verbose: bool = False) -> Tuple[str, Dict[str, int]]:
        """
        Main preprocessing pipeline - ALL outputs in Devanagari + emoji features
        
        Pipeline:
        1. EXTRACT emoji features (before anything)
        2. DETECT script (emojis ignored)
        3. PROCESS based on script
        4. REPLACE emojis (after conversion)
        5. NORMALIZE
        
        Args:
            text: Input text
            verbose: Print processing steps
        
        Returns:
            Tuple of (preprocessed_text, emoji_features)
        """
        if not isinstance(text, str) or not text.strip():
            return "", {
                'has_hate_emoji': 0, 'has_mockery_emoji': 0, 'has_positive_emoji': 0,
                'has_sadness_emoji': 0, 'has_fear_emoji': 0, 'has_disgust_emoji': 0,
                'hate_emoji_count': 0, 'mockery_emoji_count': 0, 'positive_emoji_count': 0,
                'sadness_emoji_count': 0, 'fear_emoji_count': 0, 'disgust_emoji_count': 0,
                'total_emoji_count': 0, 'hate_to_positive_ratio': 0.0, 'has_mixed_sentiment': 0
            }

        original_text = text
        
        if verbose:
            print(f"📥 Original: {original_text}")

        # =====================================================================
        # STEP 1: EXTRACT EMOJI FEATURES (BEFORE any processing)
        # =====================================================================
        emoji_features = extract_emoji_features(original_text)
        
        if verbose:
            print(f"😊 Emoji Features Extracted:")
            print(f"   Hate emojis: {emoji_features['hate_emoji_count']}")
            print(f"   Mockery emojis: {emoji_features['mockery_emoji_count']}")
            print(f"   Positive emojis: {emoji_features['positive_emoji_count']}")
            print(f"   Total emojis: {emoji_features['total_emoji_count']}")

        # =====================================================================
        # STEP 2: DETECT SCRIPT TYPE (emojis ignored for accurate detection)
        # =====================================================================
        script_type, detection_details = detect_script_type(text)
        
        if verbose:
            print(f"🔍 Detected Script: {script_type.upper()} "
                  f"(confidence: {detection_details.get('confidence', 0):.2%})")

        # =====================================================================
        # STEP 3: BASIC CLEANING (before script processing)
        # =====================================================================
        text = clean_text_basic(text)
        if verbose:
            print(f"🧹 After cleaning: {text}")

        # =====================================================================
        # STEP 4: SCRIPT-SPECIFIC PROCESSING
        # =====================================================================
        
        if script_type == "devanagari":
            processed = text
            # Translate embedded English fragments
            if self.translate_english:
                processed = translate_latin_spans(processed, self.translator)
            if verbose:
                print("✅ Devanagari → translated embedded English")
        
        elif script_type == "romanized_nepali":
            if verbose:
                print("🔄 Romanized Nepali → transliterating to Devanagari")
            processed = transliterate_romanized_nepali(text)
            if verbose:
                print(f"   After transliteration: {processed}")
        
        elif script_type == "english":
            if self.translate_english:
                if verbose:
                    print("🌐 English → translating to Devanagari")
                processed = self.translator.translate(text, fallback_to_original=True)
                if verbose:
                    print(f"   After translation: {processed}")
            else:
                if verbose:
                    print("⚠️ Translation disabled, keeping original")
                processed = text
        
        elif script_type == "mixed":
            if verbose:
                print("🔀 Mixed script → transliterate + translate")

            processed = transliterate_romanized_nepali(text)

            if self.translate_english:
                processed = translate_latin_spans(processed, self.translator)

        
        else:
            processed = text
            if verbose:
                print("❓ Unknown script → keeping as-is")

        # =====================================================================
        # STEP 5: EMOJI REPLACEMENT (AFTER script conversion to Devanagari)
        # =====================================================================
        if verbose:
            print("😊 Replacing emojis with Nepali text...")
        
        processed = replace_emojis_semantic(processed)
        
        if verbose:
            print(f"   After emoji replacement: {processed}")

        # =====================================================================
        # STEP 6: NORMALIZATION
        # =====================================================================
        final = normalize_devanagari(processed)
        final = re.sub(r"\s+", " ", final).strip()

        # =====================================================================
        # STEP 7: OPTIONAL - Inject emoji tags (Pattern 2)
        # =====================================================================
        if self.use_emoji_tags:
            final = inject_emoji_tags(final, emoji_features)
            if verbose:
                print(f"🏷️  Emoji tags injected")

        if verbose:
            print(f"✅ Final (Devanagari): {final}")
            print(f"📊 Emoji Features: {emoji_features}")
            print("-" * 80)

        return final, emoji_features
    
    def preprocess_batch(self, texts: list, verbose: bool = False) -> Tuple[list, list]:
        """
        Preprocess multiple texts efficiently
        
        Returns:
            Tuple of (preprocessed_texts, emoji_features_list)
        """
        results = [self.preprocess(text, verbose=verbose) for text in texts]
        texts_processed = [r[0] for r in results]
        features = [r[1] for r in results]
        return texts_processed, features
    
    def get_stats(self) -> dict:
        """Get preprocessing statistics"""
        stats = {
            'model_type': self.model_type,
            'translation_enabled': self.translate_english,
            'transliteration_available': TRANSLITERATION_AVAILABLE,
            'emoji_tags_enabled': self.use_emoji_tags
        }
        if self.translator:
            stats['cache_info'] = self.translator.get_cache_info()
        return stats


# ============================================================================
# CONVENIENCE FUNCTIONS
# ============================================================================

def preprocess_for_model(
    text: str,
    model_type: Literal["xlmr", "mbert", "nepalibert"] = "xlmr",
    translate_english: bool = True,
    use_emoji_tags: bool = False
) -> Tuple[str, Dict[str, int]]:
    """
    Quick preprocessing function for single text
    ALL models receive Devanagari output + emoji features
    """
    preprocessor = HateSpeechPreprocessor(
        model_type=model_type,
        translate_english=translate_english,
        use_emoji_tags=use_emoji_tags
    )
    return preprocessor.preprocess(text)


# ============================================================================
# TESTING
# ============================================================================

if __name__ == "__main__":
    print("=" * 80)
    print("PREPROCESSING WITH EMOJI INTELLIGENCE")
    print("=" * 80)
    
    test_samples = [
        ("यो नेपाली पाठ हो 🙏", "Pure Devanagari + Positive Emoji"),
        ("ma khusi xu 😊👍", "Romanized Nepali + Positive Emojis"),
        ("This is hate speech 🖕😡", "English + Hate Emojis"),
        ("You stupid मूर्ख 😤🙄", "Mixed + Mockery Emojis"),
        ("I am angry 😡🖕💣", "English + Multiple Hate Emojis"),
        ("😂😡👍", "Mixed sentiment emojis only"),
    ]
    
    print("\n" + "=" * 80)
    print("PATTERN 1: Feature Extraction (Recommended)")
    print("=" * 80)
    
    preprocessor1 = HateSpeechPreprocessor(
        model_type="xlmr",
        translate_english=True,
        use_emoji_tags=False  # Pattern 1
    )
    
    for text, description in test_samples:
        print(f"\n{'='*80}")
        print(f"Test: {description}")
        processed_text, features = preprocessor1.preprocess(text, verbose=True)
        print(f"{'='*80}")
    
    print("\n" + "=" * 80)
    print("PATTERN 2: Emoji Tag Injection (Alternative)")
    print("=" * 80)
    
    preprocessor2 = HateSpeechPreprocessor(
        model_type="xlmr",
        translate_english=True,
        use_emoji_tags=True  # Pattern 2
    )
    
    for text, description in test_samples[:3]:  # Show 3 examples
        print(f"\n{description}:")
        print(f"Input: {text}")
        processed_text, features = preprocessor2.preprocess(text, verbose=False)
        print(f"Output: {processed_text}")
        print()
    
    print("=" * 80)
    print("✅ EMOJI INTELLIGENCE IMPLEMENTED:")
    print("   Pattern 1: Extract features → Use in classifier")
    print("   Pattern 2: Inject tags → Transformer learns context")
    print("=" * 80)

PREPROCESSING WITH EMOJI INTELLIGENCE

PATTERN 1: Feature Extraction (Recommended)

Test: Pure Devanagari + Positive Emoji
📥 Original: यो नेपाली पाठ हो 🙏
😊 Emoji Features Extracted:
   Hate emojis: 0
   Mockery emojis: 0
   Positive emojis: 1
   Total emojis: 1
🔍 Detected Script: DEVANAGARI (confidence: 185.71%)
🧹 After cleaning: यो नेपाली पाठ हो 🙏
✅ Devanagari → translated embedded English
😊 Replacing emojis with Nepali text...
   After emoji replacement: यो नेपाली पाठ हो  नमस्कार 
✅ Final (Devanagari): यो नेपाली पाठ हो नमस्कार
📊 Emoji Features: {'has_hate_emoji': 0, 'has_mockery_emoji': 0, 'has_positive_emoji': 1, 'has_sadness_emoji': 0, 'has_fear_emoji': 0, 'has_disgust_emoji': 0, 'hate_emoji_count': 0, 'mockery_emoji_count': 0, 'positive_emoji_count': 1, 'sadness_emoji_count': 0, 'fear_emoji_count': 0, 'disgust_emoji_count': 0, 'total_emoji_count': 1, 'hate_to_positive_ratio': 0.0, 'has_mixed_sentiment': 0}
---------------------------------------------------------------------------

In [23]:
"""
Apply Enhanced Nepali Hate Speech Preprocessing
and save processed datasets for LOCAL training
WITH EMOJI INTELLIGENCE FEATURES
"""

import os
import pandas as pd
from tqdm import tqdm

# ------------------------------------------------------------------
# IMPORT YOUR FIXED PREPROCESSOR
# ------------------------------------------------------------------
# from preprocessing import HateSpeechPreprocessor

# ------------------------------------------------------------------
# PATHS
# ------------------------------------------------------------------
RAW_DATA_DIR = r"D:\major project\data"
OUTPUT_DIR = r"D:\major project\data\processed"

TRAIN_FILE = r"D:\major project\data\train_final.json"
VAL_FILE   = r"D:\major project\data\val_final.json"
TEST_FILE  = r"D:\major project\data\test.json"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------------------------
# LOAD DATA
# ------------------------------------------------------------------
print("📥 Loading datasets...")

train_df = pd.read_json(TRAIN_FILE)
val_df   = pd.read_json(VAL_FILE)
test_df  = pd.read_json(TEST_FILE)

print(f"✓ Train samples: {len(train_df)}")
print(f"✓ Val samples:   {len(val_df)}")
print(f"✓ Test samples:  {len(test_df)}")

# ------------------------------------------------------------------
# INITIALIZE PREPROCESSOR
# ------------------------------------------------------------------
preprocessor = HateSpeechPreprocessor(
    model_type="xlmr",
    translate_english=True,
    cache_size=2000,
    use_emoji_tags=False  # Pattern 1 (recommended)
)

# ------------------------------------------------------------------
# APPLY PREPROCESSING
# ------------------------------------------------------------------
def apply_preprocessing(df: pd.DataFrame, split_name: str) -> pd.DataFrame:
    print(f"\n🔄 Preprocessing {split_name} set...")

    processed_texts = []
    emoji_feature_rows = []

    for text in tqdm(df["Comment"].astype(str).tolist()):
        processed_text, emoji_features = preprocessor.preprocess(text)
        processed_texts.append(processed_text)
        emoji_feature_rows.append(emoji_features)

    df = df.copy()
    df["Comment_Processed"] = processed_texts

    # Expand emoji features into columns
    emoji_features_df = pd.DataFrame(emoji_feature_rows)
    df = pd.concat([df.reset_index(drop=True), emoji_features_df], axis=1)

    return df


train_df = apply_preprocessing(train_df, "TRAIN")
val_df   = apply_preprocessing(val_df, "VALIDATION")
test_df  = apply_preprocessing(test_df, "TEST")

# ------------------------------------------------------------------
# SAVE PROCESSED DATA
# ------------------------------------------------------------------
train_out = os.path.join(OUTPUT_DIR, "train_final_processed.json")
val_out   = os.path.join(OUTPUT_DIR, "val_final_processed.json")
test_out  = os.path.join(OUTPUT_DIR, "test_processed.json")

train_df.to_json(train_out, orient="records", force_ascii=False, indent=2)
val_df.to_json(val_out, orient="records", force_ascii=False, indent=2)
test_df.to_json(test_out, orient="records", force_ascii=False, indent=2)

print("\n💾 Processed files saved:")
print(f"✓ {train_out}")
print(f"✓ {val_out}")
print(f"✓ {test_out}")

# ------------------------------------------------------------------
# SANITY CHECK
# ------------------------------------------------------------------
print("\n🔍 SANITY CHECK (first 3 samples):")
for i in range(3):
    print("-" * 80)
    print(f"Original : {train_df.iloc[i]['Comment']}")
    print(f"Processed: {train_df.iloc[i]['Comment_Processed']}")
    print(f"Emoji features: "
          f"Hate={train_df.iloc[i]['has_hate_emoji']}, "
          f"Mockery={train_df.iloc[i]['has_mockery_emoji']}, "
          f"Positive={train_df.iloc[i]['has_positive_emoji']}")

# ------------------------------------------------------------------
# CACHE STATS
# ------------------------------------------------------------------
stats = preprocessor.get_stats()

print("\n📊 Preprocessing Stats:")
print(f"Translation enabled: {stats['translation_enabled']}")
print(f"Transliteration available: {stats['transliteration_available']}")
print(f"Emoji tags enabled: {stats['emoji_tags_enabled']}")

if "cache_info" in stats:
    cache = stats["cache_info"]
    print(f"Cache hits: {cache['hits']}")
    print(f"Cache misses: {cache['misses']}")
    print(f"Hit rate: {cache['hit_rate']:.2%}")

print("\n✅ DATASET PREPROCESSING COMPLETE")


📥 Loading datasets...
✓ Train samples: 5577
✓ Val samples:   620
✓ Test samples:  1450

🔄 Preprocessing TRAIN set...


100%|██████████| 5577/5577 [10:52<00:00,  8.54it/s]



🔄 Preprocessing VALIDATION set...


100%|██████████| 620/620 [01:19<00:00,  7.76it/s]



🔄 Preprocessing TEST set...


100%|██████████| 1450/1450 [02:19<00:00, 10.39it/s]


💾 Processed files saved:
✓ D:\major project\data\processed\train_final_processed.json
✓ D:\major project\data\processed\val_final_processed.json
✓ D:\major project\data\processed\test_processed.json

🔍 SANITY CHECK (first 3 samples):
--------------------------------------------------------------------------------
Original : भूकने कूकूर ले कहीले टोकदैन भन्छ हो रैछ साला धोती तीमी हरू लाई केको अजादी जाहीयो
Processed: भूकने कूकूर ले कहीले टोकदैन भन्छ हो रैछ साला धोती तीमी हरू लाई केको अजादी जाहीयो
Emoji features: Hate=0, Mockery=0, Positive=0
--------------------------------------------------------------------------------
Original : तेरी आमाको पसल मूजी कूकूरको मूत खाएसी जे पनी सून्छ ऋतुजितको जाँठले जा ऋतुजितको झोल खान
Processed: तेरी आमाको पसल मूजी कूकूरको मूत खाएसी जे पनी सून्छ ऋतुजितको जाँठले जा ऋतुजितको झोल खान
Emoji features: Hate=0, Mockery=0, Positive=0
--------------------------------------------------------------------------------
Original : thukka ta machikne valu kun level ko ho